In [17]:
from pprint import pprint
import random
import numpy as np
import tensorflow as tf

from catanatron.game import Game
from catanatron.models.player import Player, RandomPlayer, Color
from catanatron.players.weighted_random import WeightedRandomPlayer
from catanatron_experimental.play import play_batch
from catanatron_gym.envs.catanatron_env import to_action_space, ACTION_SPACE_SIZE, from_action_space
from catanatron_gym.features import create_sample_vector

path = "data/models/ab2-copycat-model"
model = tf.keras.models.load_model(path)


class MyPlayer(Player):
    def decide(self, game, playable_actions):
        """Should return one of the playable_actions.

        Args:
            game (Game): complete game state. read-only.
            playable_actions (Iterable[Action]): options to choose from
        Return:
            action (Action): Chosen element of playable_actions
        """
        # ===== YOUR CODE HERE =====
        action_ints = [to_action_space(a) for a in playable_actions]
        mask = np.zeros(ACTION_SPACE_SIZE, dtype=np.float)
        mask[action_ints] = 1
        mask[mask == 0] = np.nan

        # Get action probabilities with neural network.
#         TODO: Ensure same ordering as in training...
        record = create_sample_vector(game, self.color)
        X = [record]
        result = model.call(tf.convert_to_tensor(X))

        # Multiply mask with output, and take max.
        clipped_probabilities = np.multiply(mask, result[0])
        clipped_probabilities[np.isnan(clipped_probabilities)] = -np.inf
        action_index = np.argmax(clipped_probabilities)
        action = from_action_space(action_index, playable_actions)
        
        return action
        # ===== END YOUR CODE =====

# Play a simple 4v4 game. Edit MyPlayer with your logic!
players = [
    MyPlayer(Color.RED),
    RandomPlayer(Color.WHITE),
]
wins, results_by_player, games = play_batch(100, players)

pprint(wins)
pprint(results_by_player)

2022-01-09 22:45:20,827 DEBUG Playing game 1 / 100. Seating: [MyPlayer:RED, RandomPlayer:WHITE]
2022-01-09 22:45:22,346 DEBUG {'MyPlayer:RED': 4, 'RandomPlayer:WHITE': 10} (1.52 secs) [Color.WHITE:567(1399)]
2022-01-09 22:45:22,350 DEBUG Playing game 2 / 100. Seating: [RandomPlayer:WHITE, MyPlayer:RED]
2022-01-09 22:45:23,055 DEBUG {'MyPlayer:RED': 11, 'RandomPlayer:WHITE': 2} (0.705 secs) [Color.RED:279(751)]
2022-01-09 22:45:23,057 DEBUG Playing game 3 / 100. Seating: [MyPlayer:RED, RandomPlayer:WHITE]
2022-01-09 22:45:23,715 DEBUG {'MyPlayer:RED': 4, 'RandomPlayer:WHITE': 10} (0.657 secs) [Color.WHITE:339(840)]
2022-01-09 22:45:23,718 DEBUG Playing game 4 / 100. Seating: [RandomPlayer:WHITE, MyPlayer:RED]
2022-01-09 22:45:24,267 DEBUG {'MyPlayer:RED': 7, 'RandomPlayer:WHITE': 11} (0.548 secs) [Color.WHITE:208(580)]
2022-01-09 22:45:24,270 DEBUG Playing game 5 / 100. Seating: [RandomPlayer:WHITE, MyPlayer:RED]
2022-01-09 22:45:24,680 DEBUG {'MyPlayer:RED': 2, 'RandomPlayer:WHITE': 10

{<Color.RED: 'RED'>: 51, <Color.WHITE: 'WHITE'>: 44}
{<Color.RED: 'RED'>: [4,
                      11,
                      4,
                      7,
                      2,
                      10,
                      7,
                      10,
                      4,
                      10,
                      5,
                      10,
                      6,
                      10,
                      4,
                      11,
                      10,
                      4,
                      2,
                      10,
                      10,
                      10,
                      5,
                      10,
                      4,
                      4,
                      7,
                      11,
                      10,
                      11,
                      10,
                      11,
                      6,
                      4,
                      7,
                      10,
                      10,
   